# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("huyndaiShip").getOrCreate()

23/09/12 10:56:40 WARN Utils: Your hostname, ubuntu-System-Product-Name resolves to a loopback address: 127.0.1.1; using 192.168.1.161 instead (on interface enp6s0)
23/09/12 10:56:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/12 10:56:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.csv("cruise_ship_info.csv", header=True, inferSchema=True)

In [4]:
df.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [5]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [6]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="Cruise_line", outputCol="Cruise_line_Index")
indexed_df = indexer.fit(df).transform(df)
indexed_df.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_line_Index|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|             16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|             16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|              1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|              1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|              1.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|       

In [7]:
indexed_df.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_line_Index']

In [8]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(
    inputCols=['Cruise_line_Index','Age','Tonnage','passengers','length','cabins','passenger_density'],
    outputCol="features")

new_df = assembler.transform(indexed_df)
new_df.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----------------+--------------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_line_Index|            features|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----------------+--------------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|             16.0|[16.0,6.0,30.2769...|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|             16.0|[16.0,6.0,30.2769...|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|              1.0|[1.0,26.0,47.262,...|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|              1.0|[1.0,11.0,110.0,2...|
|    Destiny|   Carnival| 17|           101.353|

In [10]:
final_df = new_df.select("crew","features")
final_df.show()

+----+--------------------+
|crew|            features|
+----+--------------------+
|3.55|[16.0,6.0,30.2769...|
|3.55|[16.0,6.0,30.2769...|
| 6.7|[1.0,26.0,47.262,...|
|19.1|[1.0,11.0,110.0,2...|
|10.0|[1.0,17.0,101.353...|
| 9.2|[1.0,22.0,70.367,...|
| 9.2|[1.0,15.0,70.367,...|
| 9.2|[1.0,23.0,70.367,...|
| 9.2|[1.0,19.0,70.367,...|
|11.5|[1.0,6.0,110.2389...|
|11.6|[1.0,10.0,110.0,2...|
| 6.6|[1.0,28.0,46.052,...|
| 9.2|[1.0,18.0,70.367,...|
| 9.2|[1.0,17.0,70.367,...|
| 9.3|[1.0,11.0,86.0,21...|
|11.6|[1.0,8.0,110.0,29...|
|10.3|[1.0,9.0,88.5,21....|
| 9.2|[1.0,15.0,70.367,...|
| 9.3|[1.0,12.0,88.5,21...|
| 9.2|[1.0,20.0,70.367,...|
+----+--------------------+
only showing top 20 rows



In [11]:
train_data,test_data = final_df.randomSplit([0.7,0.3])
train_data.show()

+----+--------------------+
|crew|            features|
+----+--------------------+
|0.59|[9.0,22.0,3.341,0...|
|0.59|[9.0,22.0,3.341,0...|
|0.88|[15.0,25.0,5.35,1...|
|0.88|[15.0,27.0,5.35,1...|
|1.46|[10.0,27.0,12.5,3...|
| 1.6|[14.0,21.0,10.0,2...|
| 1.6|[14.0,24.0,10.0,2...|
| 1.8|[15.0,23.0,14.745...|
|1.97|[11.0,19.0,16.8,2...|
| 2.1|[11.0,19.0,16.8,2...|
|2.11|[10.0,16.0,19.2,3...|
|2.87|[11.0,12.0,25.0,3...|
|2.95|[11.0,13.0,25.0,3...|
|2.97|[7.0,36.0,16.852,...|
|3.24|[10.0,14.0,33.0,4...|
| 3.5|[19.0,48.0,22.08,...|
|3.55|[16.0,6.0,30.2769...|
|3.73|[2.0,14.0,30.2769...|
|3.73|[2.0,14.0,30.2769...|
|3.85|[5.0,23.0,25.0,7....|
+----+--------------------+
only showing top 20 rows



In [12]:
test_data.show()

+----+--------------------+
|crew|            features|
+----+--------------------+
| 0.6|[6.0,12.0,2.329,0...|
| 1.6|[14.0,27.0,10.0,2...|
|3.55|[16.0,6.0,30.2769...|
| 3.8|[4.0,40.0,28.0,11...|
|4.38|[4.0,21.0,38.0,10...|
| 4.6|[3.0,25.0,38.0,7....|
| 4.7|[9.0,21.0,19.093,...|
| 5.3|[3.0,29.0,33.92,1...|
|5.88|[3.0,19.0,55.451,...|
| 6.0|[3.0,13.0,61.0,13...|
|6.17|[5.0,22.0,52.926,...|
| 6.3|[4.0,25.0,42.0,15...|
|6.36|[5.0,27.0,53.872,...|
|6.71|[0.0,23.0,48.563,...|
| 7.2|[0.0,18.0,70.0,18...|
| 7.5|[9.0,24.0,40.0530...|
|7.65|[0.0,16.0,78.491,...|
|7.66|[5.0,17.0,75.166,...|
|8.22|[0.0,22.0,73.941,...|
|8.42|[3.0,10.0,81.7689...|
+----+--------------------+
only showing top 20 rows



In [13]:
unlabeled_data = test_data.select('features')
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[6.0,12.0,2.329,0...|
|[14.0,27.0,10.0,2...|
|[16.0,6.0,30.2769...|
|[4.0,40.0,28.0,11...|
|[4.0,21.0,38.0,10...|
|[3.0,25.0,38.0,7....|
|[9.0,21.0,19.093,...|
|[3.0,29.0,33.92,1...|
|[3.0,19.0,55.451,...|
|[3.0,13.0,61.0,13...|
|[5.0,22.0,52.926,...|
|[4.0,25.0,42.0,15...|
|[5.0,27.0,53.872,...|
|[0.0,23.0,48.563,...|
|[0.0,18.0,70.0,18...|
|[9.0,24.0,40.0530...|
|[0.0,16.0,78.491,...|
|[5.0,17.0,75.166,...|
|[0.0,22.0,73.941,...|
|[3.0,10.0,81.7689...|
+--------------------+
only showing top 20 rows



In [16]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol='features', labelCol='crew', predictionCol='prediction',maxIter=10, regParam=0.3)


# Fit the model
correct_model = lr.fit(train_data)

# Print the coefficients and intercept for linear regression
print("Coefficients: %s" % str(correct_model.coefficients))
print("Intercept: %s" % str(correct_model.intercept))

# Summarize the model over the training set and print out some metrics
trainingSummary = correct_model.summary
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

Coefficients: [0.005428024893236106,-0.012341280341969066,0.017971456518578722,0.040107016246867175,0.39757710584566336,0.34259353753743826,0.0025065778061329246]
Intercept: -0.48748478327127526
numIterations: 0
objectiveHistory: [0.0]
+--------------------+
|           residuals|
+--------------------+
|-0.13555150143923822|
| -0.1395272724976947|
| -0.6526511676367702|
| -0.6270162668078881|
|-0.27069795523144746|
|-0.21863064271067634|
|-0.18160680168476917|
| -1.0060914198745021|
| -0.4812345175379822|
|-0.35123451753798207|
| -0.4708442787523972|
| -0.3586953552233907|
|-0.26650436326886506|
| -0.3282908971537353|
| -0.1782039605417567|
|-0.07237897849501662|
|-0.48247595936960863|
|-0.08664361335042514|
| -0.0871198985605659|
|-0.04242534253791419|
+--------------------+
only showing top 20 rows

RMSE: 1.066438
r2: 0.902819
